
# Financial Instruments Final Exam (2024) Solutions

This notebook solves the five questions from the provided final exam. Every question is approached in order with calculations, explanations, data frames, and Plotly visualizations.


In [1]:

import math
from dataclasses import asdict
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from scipy.stats import norm

from final_exam_utils import (
    forward_rate,
    currency_swap_fixed_rate,
    currency_swap_value,
    merton_model,
    senior_subordinated_values,
    european_call_binomial,
    american_call_with_dividend,
    structured_product_price,
    structured_product_delta_beta,
)

pd.set_option('display.float_format', lambda x: f"{x:,.4f}")



## Question 1 — Short answers

*Advantages of Monte Carlo vs. binomial; call vs. put parity at-the-money; why FX futures are poor predictors.*


In [2]:

concepts = pd.DataFrame(
    {
        "Aspect": [
            "Path-dependence / dimensionality",
            "Accuracy for early exercise",
            "Convergence speed",
            "Hedging information",
        ],
        "Monte Carlo": [
            "Handles many risk factors and exotic path features naturally",
            "Least-squares or regression methods needed for American features",
            "Slow for low-variance estimates; standard error ~ 1/sqrt(N)",
            "Greeks via pathwise or likelihood ratio methods",
        ],
        "Binomial": [
            "Tree explodes with many factors; best for 1 underlying",
            "Built-in early exercise logic at nodes",
            "Fast convergence for vanilla payoffs as steps increase",
            "Delta/hedge ratios visible at each node",
        ],
    }
)
concepts


,Aspect,Monte Carlo,Binomial
0,Path-dependence / dimensionality,Handles many risk factors and exotic path feat...,Tree explodes with many factors; best for 1 un...
1,Accuracy for early exercise,Least-squares or regression methods needed for...,Built-in early exercise logic at nodes
2,Convergence speed,Slow for low-variance estimates; standard erro...,Fast convergence for vanilla payoffs as steps ...
3,Hedging information,Greeks via pathwise or likelihood ratio methods,Delta/hedge ratios visible at each node



* A European call and put struck at the money share the same forward price; with no dividends and equal maturity, put–call parity implies similar values. Small differences arise from convexity of payoffs and interest effects, but the call is not systematically higher.
* Futures on FX embed the interest-rate differential and a risk premium; they reflect cost-of-carry rather than the market's unbiased expectation, so they are unreliable predictors of the future spot.


## Question 2 — FX forwards and currency swap

In [3]:

S0 = 1.1
rd = 0.04
rf = 0.05
maturities = [0.5, 1.0, 1.5]
forwards = [forward_rate(S0, rd, rf, t) for t in maturities]
forward_df = pd.DataFrame({"Maturity (yrs)": maturities, "Forward USD/EUR": forwards})
forward_df


,Maturity (yrs),Forward USD/EUR
0,0.5000,1.0945
1,1.0000,1.0891
2,1.5000,1.0836


In [4]:

cashflows_eur = [2.5, 2.5, 102.5]  # coupons and principal
k_swap = currency_swap_fixed_rate(S0, rd, rf, cashflows_eur, maturities)

# Contract in part (c): coupons at k_swap, principal at spot
value_principal_at_spot = currency_swap_value([100], [1.5], rd, k_swap, S0)

# Re-valuation when rates align at 4% (forwards collapse to spot)
new_forward = S0  # because rd = rf
mark_to_market = currency_swap_value(cashflows_eur, maturities, 0.04, k_swap, new_forward)

summary_q2 = pd.DataFrame(
    {
        "Item": ["Swap rate K (USD per EUR)", "PV benefit of spot principal", "MTM after rates = 4%"],
        "USD": [k_swap, value_principal_at_spot, mark_to_market],
    }
)
summary_q2


,Item,USD
0,Swap rate K (USD per EUR),1.0840
1,PV benefit of spot principal,1.5054
2,MTM after rates = 4%,1.6206


In [5]:

fig = px.line(forward_df, x="Maturity (yrs)", y="Forward USD/EUR", markers=True, title="Forward curve")
fig


## Question 3 — Merton structural model

In [6]:

asset_value = 500
asset_vol = 0.25
r = 0.02
T = 7
face_value = 400
shares = 500_000

merton_res = merton_model(asset_value, face_value, asset_vol, r, T, shares)

results = pd.DataFrame(
    {
        "Metric": [
            "Total equity value (million)",
            "Share price ($)",
            "Equity volatility",
            "Debt value (million)",
            "Debt yield to maturity",
            "Risk-neutral default probability",
            "Debt risk premium",
        ],
        "Value": [
            merton_res.equity_value,
            merton_res.share_price,
            merton_res.equity_vol,
            merton_res.debt_value,
            merton_res.debt_yield,
            merton_res.default_prob,
            merton_res.debt_yield - r,
        ],
    }
)
results


,Metric,Value
0,Total equity value (million),201.3322
1,Share price ($),0.0004
2,Equity volatility,0.5032
3,Debt value (million),298.6678
4,Debt yield to maturity,0.0417
5,Risk-neutral default probability,0.4136
6,Debt risk premium,0.0217


In [7]:

senior_face = face_value / 2
sub_face = face_value / 2
senior_val, sub_val = senior_subordinated_values(asset_value, senior_face, sub_face, asset_vol, r, T)

senior_yield = -math.log(senior_val / senior_face) / T
sub_yield = -math.log(sub_val / sub_face) / T

tranche_df = pd.DataFrame(
    {
        "Tranche": ["Senior (200 face)", "Subordinated (200 face)"],
        "Value": [senior_val, sub_val],
        "Yield": [senior_yield, sub_yield],
        "Risk premium vs rf": [senior_yield - r, sub_yield - r],
    }
)
tranche_df


,Tranche,Value,Yield,Risk premium vs rf
0,Senior (200 face),178.2568,0.0164,-0.0036
1,Subordinated (200 face),129.1813,0.0624,0.0424


## Question 4 — Binomial option pricing with dividend

In [8]:

S0 = 100
u = 1.2
d = 1/1.2
K = 100
r = 0.02

p = (math.exp(r) - d) / (u - d)
call_euro_2y = european_call_binomial(S0, K, r, u, d, 2)
call_am_2y = call_euro_2y  # no dividends

price_am_div, stock_tree = american_call_with_dividend(S0, K, r, u, d, 3, dividend_time=2, dividend_amount=10, threshold=100)

pd.DataFrame(
    {
        "Quantity": ["Risk-neutral p", "European call (2y)", "American call (2y)", "American call with dividend (3y)"],
        "Value": [p, call_euro_2y, call_am_2y, price_am_div],
    }
)


,Quantity,Value
0,Risk-neutral p,0.5096
1,European call (2y),10.9801
2,American call (2y),10.9801
3,American call with dividend (3y),10.9505


In [9]:

# Inspect node values at year 2 after dividend to see exercise logic
node_year2 = pd.DataFrame({"State": ["dd", "du", "ud", "uu"], "Price after dividend": stock_tree[2]})
node_year2


,State,Price after dividend
0,dd,69.4444
1,du,90.0000
2,ud,90.0000
3,uu,134.0000


In [10]:

terminal_prices = np.linspace(50, 170, 200)
payoff = np.maximum(terminal_prices - K, 0)
fig = go.Figure()
fig.add_trace(go.Scatter(x=terminal_prices, y=payoff, name="Call payoff"))
fig.update_layout(title="Call payoff at maturity", xaxis_title="Stock price", yaxis_title="Payoff")
fig


## Question 5 — Structured product pricing and hedging

In [11]:

S0 = 1000
r = 0.02
T = 1
put_1000 = 50
put_1200 = 188
K1, K2 = 1000, 1200

product_price, call1, call2 = structured_product_price(S0, r, T, put_1000, put_1200, K1, K2)

summary = pd.DataFrame(
    {
        "Item": ["Call strike 1000", "Call strike 1200", "Structured product price"],
        "Value": [call1, call2, product_price],
    }
)
summary


,Item,Value
0,Call strike 1000,69.8013
1,Call strike 1200,11.7616
2,Structured product price,"1,096.2781"


In [12]:

# Payoff diagram
index_prices = np.linspace(600, 1600, 250)
returns = (index_prices - S0) / S0
payoff = []
for s, R in zip(index_prices, returns):
    if R < 0:
        payoff.append(1000)
    elif R <= 0.2:
        payoff.append(1000 + 2000 * R)
    else:
        payoff.append(1400)

fig = go.Figure()
fig.add_trace(go.Scatter(x=index_prices, y=payoff, name="Structured payoff"))
fig.update_layout(title="Structured product payoff", xaxis_title="Index level at maturity", yaxis_title="Payoff ($)")
fig


In [13]:

# Black-Scholes deltas for hedging
sigma = 0.30

def call_price_bs(S, K, r, sigma, T):
    d1 = (math.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    return S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)

call1_theory = call_price_bs(S0, K1, r, sigma, T)
call2_theory = call_price_bs(S0, K2, r, sigma, T)
product_bs = math.exp(-r * T) * 1000 + 2 * call1_theory - 2 * call2_theory

# hedge
product_delta_1000, beta_1000 = structured_product_delta_beta(S0, r, sigma, T, K1, K2)
product_delta_1300, beta_1300 = structured_product_delta_beta(1300, r, sigma, T, K1, K2)

hedge_df = pd.DataFrame(
    {
        "Underlying": ["1000", "1300"],
        "Product delta": [product_delta_1000, product_delta_1300],
        "Beta approx": [beta_1000, beta_1300],
    }
)

pd.DataFrame(
    {
        "Item": ["BS call 1000", "BS call 1200", "BS structured value"],
        "Value": [call1_theory, call2_theory, product_bs],
    }
), hedge_df


(                  Item      Value
 0         BS call 1000   128.2158
 1         BS call 1200    59.9757
 2  BS structured value 1,116.6789,
   Underlying  Product delta  Beta approx
 0       1000         0.4758       0.3467
 1       1300         0.3536       0.3350)